In [ ]:
# Getting Started
!uv pip install pynetlogo matplotlib seaborn pandas

# Set the path to your NetLogo installation
# Note! This is OS and machine dependent.
# Perhaps someone has an NL docker image out there?
# See https://pynetlogo.readthedocs.io/en/latest/_docs/pynetlogo.html#pynetlogo.core.NetLogoLink


# Module 2 Assignment: Schelling Segregation Model

*CAS 520, Peter Dresslar*

For this assignment we will use [pynetlogo](https://pynetlogo.readthedocs.io/) to capture and report simulation outputs. An excellent and relevant example appears [here](https://pynetlogo.readthedocs.io/en/latest/_docs/introduction.html)

**Important** pynetlogo will apparently only work reliably with NetLogo 6.3.0. While that presents challenges of its own, it is also important to note the the model we are using, `segregation`, *must* have a version number that matches that version of NetLogo. So, grabbing the latest version of the model may not work without tweaking the file. The model with adjusted version number is in this repository.

`pynetlogo` requires a Java install, for which I used `brew install cask temurin` (on Apple silicon). While the jvm path can be overridden, it appears that pynetlogoʻs implementation of JPype works to automatially find the installed JVM from that cask. Note that I wound up having to point to the `app` directory within the Netlogo base installation.

## Step 1: First Analysis

### Experiment 1: Set density to 80% and similarity wanted to 30% and run the simulation several times.

### Experiment 2: Keeping density at 80%, change the similarity threshold to 90%.

### Experiment 3: While the model is still running from step 2, slowly move the similarity slider to the left and note when the model behaviour changes and segregation emerges.

### Experiment 4: Rerun the simulation a few times with 80% density and similarity threshold at the tipping point you discovered.


We will begin Step 1 by building our experiment harness:

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynetlogo
import os
import subprocess
import platform

base_model = "./Segregation.nlogo" # (copied into this directory!)

def initialize_netlogo():

    ### Some unplesantness for pynetlogo setup:
    # Get Java home (should return ARM Java if that's what's installed)
    java_home = subprocess.check_output(['/usr/libexec/java_home']).decode().strip()
    jvm_path = os.path.join(java_home, 'lib', 'server', 'libjvm.dylib')
    netlogo_path = '/Users/peterdresslar/Workspace/NetLogo-6.3.0/app'

    # Print for verification
    print(f"Python architecture: {platform.machine()}")
    print(f"Using JVM at: {jvm_path}")
    print(f"Using NetLogo at: {netlogo_path}")

    # Get a netlogo instance
    netlogo = pynetlogo.NetLogoLink(
        gui=False,  # cannot set to true for macs
        netlogo_home=netlogo_path,
        # jvm_path=jvm_path
    )

    return netlogo

def find_breakpoint(netlogo, model, precision):
    """
    This is tricky! We start with the given density and similarity wanted.
    We know from manual testing that there is a breakpoint at which point the model 
    will reduce to 0% unhappy. We also can observe that this "collapse" occurs within
    1000 ticks, and that the go procedure will halt the model at that point.
    
    We can run a simple binary "search" to find this point,
    by running the model with similarity-wanted values between 1.0 and 90.0.

    If the model stops within 1000 ticks, we know that the breakpoint is higher than the 
    searched similarity-wanted value. If the model runs the full 1000 steps, we know that
    the breakpoint is lower than the searched similarity-wanted value.

    Since we are setting values programatically, we have the luxury of looking beyond the integer values of similarity-wanted.

    args:
        netlogo: the netlogo instance
        model: the model to run

    returns:
        breakpoint: the similarity-wanted value at which the model breaks down

    """
    density = 80.0
    similarity_wanted = 90.0
    upper_bound = similarity_wanted
    lower_bound = 1.0 # not sure how the model would deal with zeroes!
    breakpoint = None

    step_size = 10 ** (-precision)  # 10e-precision, in other words.
    
    iteration = 0
    while breakpoint is None:
        iteration += 1
        midpoint = round((upper_bound + lower_bound) / 2, 6) # round to 4 decimal places
        print(f"Iteration {iteration} at {midpoint}")
        netlogo.load_model(model)
        netlogo.command(f"set density {density}")
        netlogo.command(f"set %-similar-wanted {midpoint}")
        netlogo.command("setup")
        
        netlogo.command("repeat 1000 [go]")
        # we should be able to see if the model halted early by checking the step number 

        ticks = netlogo.report("ticks")
        print(f"Ticks: {ticks}")

        if ticks < 1000:
            lower_bound = midpoint + step_size
        else:
            upper_bound = midpoint - step_size

        if upper_bound - lower_bound < step_size:
            breakpoint = midpoint
    
    return breakpoint
        
    
def cluster_analysis(data):
    """
    Our experiment data will include the a data frame with turtles of two colors:



    The model calls these "blue" and "orange", respectively.

    We will measure the clumpyness of the physical data using a standard pandas cluster analysis.
    """
    results = None


    return data

def run_experiment(netlogo, model, experiment_name, number_of_runs, density, percent_similar_wanted, max_ticks):
    """
    Run the model with the given parameters and return the results.

    args:
        netlogo: the netlogo instance
        model: the model to run
        experiment_name: the name of the experiment
        number_of_runs: the number of times to run the model
        density: the density of the model
        similarity_wanted: the similarity wanted of the model
        max_ticks: the maximum number of ticks to run the model

    returns:
        data: a dictionary containing experiment results
        includes:
            - number of runs
            - density
            - similarity wanted
            - average average similarity values
            - average time to happiness, in ticks (or N/A if model never halts)
            - cluster analysis results

    """
    data = {
        "experiment_name": experiment_name,
        "number_of_runs": number_of_runs,
        "density": density,
        "percent_similar_wanted": percent_similar_wanted,
        "average_similarity_values": [],
        "time_to_happiness_values": [],
        "cluster_analysis_values": [],
        "average_average_similarity_values": [],
        "average_time_to_happiness": [],
        "average_cluster_analysis_values": []
    }

    # collectors
    time_to_happiness_values = []
    average_similarity_values = []
    cluster_analysis_values = []

    for run, _ in enumerate(range(number_of_runs), 1):   # is there an easier way to do this? yes! I :heart enumerate.
        print(f"Experiment name {experiment_name}, Run {run} of {number_of_runs}")

        netlogo.load_model(model)
        
        netlogo.command(f"set density {density}")
        netlogo.command(f"set %-similar-wanted {percent_similar_wanted}")
        netlogo.command("setup")
        
        # run the model. it will halt from inside netlogo if it meets the stopping condition (no unhappy turtles)
        netlogo.command(f"repeat {max_ticks} [go]")

        ticks = netlogo.report("ticks")
        if ticks < max_ticks:   # we will assume that the model did not achieve happiness on the very last tick!
            run_time_to_happiness = ticks
        else:
            run_time_to_happiness = None

        run_average_similarity = netlogo.report("percent-similar") 
        # cluster analysis
        # turtle_data = netlogo.report("[list who xcor ycor color] of turtles")
        turtle_data = 1
        run_cluster_analysis = cluster_analysis(turtle_data)  # need to write function

        ticks_print = ticks if ticks else "N/A"   # donʻt break the print statement
        print(f" Run {run} complete, ticks: {ticks_print}, average similarity: {run_average_similarity}, cluster analysis: {run_cluster_analysis}")

        # store to collectors
        time_to_happiness_values.append(run_time_to_happiness)
        average_similarity_values.append(run_average_similarity)
        cluster_analysis_values.append(run_cluster_analysis)

    # move collectors to data
    data["time_to_happiness_values"] = time_to_happiness_values
    data["average_similarity_values"] = average_similarity_values
    data["cluster_analysis_values"] = cluster_analysis_values

    # letʻs do some post-processing
    data["average_time_to_happiness"] = sum(time_to_happiness_values) / len(time_to_happiness_values)
    data["average_average_similarity_values"] = sum(average_similarity_values) / len(average_similarity_values)
    data["average_cluster_analysis_values"] = sum(data["average_cluster_analysis_values"]) / len(data["average_cluster_analysis_values"])

    print(f"Experiment {experiment_name} complete, average time to happiness: {data['average_time_to_happiness']}, average average similarity: {data['average_average_similarity_values']}, average cluster analysis: {data['average_cluster_analysis_values']}")
    
    return data

# Since we are initializing netlogo here, you must run this cell before running the experiments.

netlogo = initialize_netlogo()


Python architecture: arm64
Using JVM at: /Library/Java/JavaVirtualMachines/temurin-24.jdk/Contents/Home/lib/server/libjvm.dylib
Using NetLogo at: /Users/peterdresslar/Workspace/NetLogo-6.3.0/app


/Users/peterdresslar/Workspace/cas-520/.venv/lib/python3.12/site-packages/pynetlogo/core.py:209: UserWarning: could not find default NetLogo extensions folder. Extensions not available
  warnings.warn(


We are ready to run our experiments! Letʻs start with Experiment 1.

In [32]:
experiment_1 = run_experiment(netlogo, base_model, experiment_name="Experiment 1", number_of_runs=2, density=80, percent_similar_wanted=30, max_ticks=1000)

Experiment name Experiment 1, Run 1 of 2
 Run 1 complete, ticks: 10.0, average similarity: 71.71296984103402, cluster analysis: 1
Experiment name Experiment 1, Run 2 of 2
 Run 2 complete, ticks: 11.0, average similarity: 72.0706021951943, cluster analysis: 1


ZeroDivisionError: division by zero